In [1]:
import numpy as np
import pandas as pd

import requests
import math
from sklearn import model_selection

import h2o
from h2o.automl import H2OAutoML

import csv
import os 

from pandas_profiling import ProfileReport

In [2]:
#!pip install xlrd  # Uncomment this, if you get errors parsing XLSX
pd.set_option('expand_frame_repr', True)
pd.set_option('max_colwidth',9999)
pd.set_option('display.max_columns',9999)
pd.set_option('display.max_rows',9999)
data_dictionary = pd.read_excel("https://resources.lendingclub.com/LCDataDictionary.xlsx")

data_dictionary

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.
5,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
6,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/credit limit for all bankcard accounts.


In [3]:
def parse_dates(x):
    return datetime.strptime(x, "%b-%d")

lc = pd.read_csv("LoanStats3c.csv.zip", skiprows=1,verbose=False, parse_dates=['issue_d'],low_memory=False) 
lc.shape

(235631, 144)

In [4]:
print(lc.loan_status.unique())
print(lc.loan_status.value_counts())

['Charged Off' 'Fully Paid' 'Current' 'In Grace Period'
 'Late (31-120 days)' nan]
Fully Paid            193803
Charged Off            41730
Current                   45
Late (31-120 days)        41
In Grace Period           10
Name: loan_status, dtype: int64


In [5]:
lc = lc[lc.loan_status.isin(['Fully Paid','Charged Off'])]
lc.loan_status.unique()

array(['Charged Off', 'Fully Paid'], dtype=object)

In [6]:
# Drop the columns from the data frame that are Target Leakage ones
# Target Leakage columns are generally created in hindsight by analysts/data engineers/operations after an outcome 
# was detected in historical data. If we don't remove them now, they would climb to the top of the feature list after a model is built and 
# falsely increase the accuracy to 95% :) 
#
# In Production or real life scoring environment, don't expect these columns to be available at scoring time
# , that is,when someone applies for a loan. So we don't train on those columns ...

ignored_cols = [ 
                'out_prncp',                 # 剩餘未償還本金金額
                'out_prncp_inv',             # 剩餘未償還本金佔總額的比例 
                'total_pymnt',               # 至今收到的已付額
                'total_pymnt_inv',           # 至今收到的款項，佔投資者資助總額的比例
                'total_rec_prncp',           # 至今收到的本金 
                'total_rec_int',             # 至今收到的利息
                'total_rec_late_fee',        # 至今為止收取的延滯金
                'recoveries',                # 過帳總回收費用
                'collection_recovery_fee',   # 郵寄回收費
                'last_pymnt_d',              # 上個月已收到付款
                'last_pymnt_amnt',           # 上一個收到的總付款金額
                'next_pymnt_d',              # 下一個預定的付款日期
                'last_credit_pull_d',        # 一個月內最近一筆取消貸款的信貸
                'settlement_term',           # 借款人將在償還計劃中使用的月數
                'settlement_date',           # 借款人同意和解計劃的日期
                'settlement_amount',         # 借款人已同意償還的貸款金額
                'settlement_percentage',     # 結算金額佔貸款還清餘額的百分比
                'settlement_status',         # 借款人的和解計劃的狀態。可能的值為：
                'debt_settlement_flag',      # 標記已清帳的借款人是否正在與債務清算公司合作。 
                'debt_settlement_flag_date'  # 設置Debt_Settlement_Flag的最新日期
    
                ]

lc = lc.drop(columns=ignored_cols, axis = 1)

In [7]:
# After dropping Target Leakage columns, we have 235K rows and 124 columns
lc.shape

(235533, 124)

### Split to Train and Test - 80%/20%  and write it to two zipped CSV files

In [8]:
print('當前的路徑為--->' + os.getcwd())

train_lc, test_lc = model_selection.train_test_split(lc, test_size=0.2, random_state=10,stratify=lc['loan_status'])
train_lc.to_csv('0217train_lc.csv', index=False,compression="zip")
test_lc.to_csv('test_lc.csv', index=False,compression="zip")

當前的路徑為--->C:\Users\justi\OneDrive\桌面\Lending club


In [9]:
# Let's look at the shape of training data
print(train_lc.shape)
print(test_lc.shape)

(188426, 124)
(47107, 124)


In [10]:
# Replace below with settings in your environment before you run the notebook

h2o3_ip = "<ip address of the H2O-3 cluster that's running>" 

In [11]:
# If you want to create and run a local instance instead, (where Jupyter kernel is running) just uncomment below:
#h2o.init()
# and comment the code below:

#h2o.init(ip=h2o3_ip,strict_version_check=False)
h2o.init()
# If things break, uninstall and install the h2o library and you set strick_version_check to True to see 
# what's going on

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)56-b12)
  Starting server from C:\Users\justi\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\justi\AppData\Local\Temp\tmpb7_cdfu6
  JVM stdout: C:\Users\justi\AppData\Local\Temp\tmpb7_cdfu6\h2o_justi_started_from_python.out
  JVM stderr: C:\Users\justi\AppData\Local\Temp\tmpb7_cdfu6\h2o_justi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,+08:00
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,13 days
H2O cluster name:,H2O_from_python_justi_rnvcmh
H2O cluster total nodes:,1
H2O cluster free memory:,3.523 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [12]:
train_lc.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,url,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
count,0.0,188426.000000,188426.000000,188426.000000,188426.000000,1.884260e+05,0.0,188426.000000,188426.000000,188426.000000,95684.000000,33192.000000,188426.000000,188426.000000,1.884260e+05,188426.000000,188426.000000,53097.000000,188426.0,0.0,0.0,0.0,188426.000000,1.884260e+05,1.884260e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.884260e+05,0.0,0.0,0.0,188426.000000,188422.000000,186474.000000,186342.000000,188426.000000,188426.000000,182717.000000,188426.000000,188426.000000,188426.000000,188426.000000,186634.000000,49713.000000,171033.000000,67722.000000,188426.000000,188426.000000,188426.000000,188426.000000,188426.000000,188426.000000,188426.000000,188426.000000,188426.000000,188426.000000,182136.000000,188426.000000,188426.000000,188426.000000,188426.000000,186390.000000,188426.000000,188426.000000,1.884260e+05,1.884260e+05,1.884260e+05,1.884260e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,344.0,344.000000,344.0,344.000000,266.000000,344.000000,344.000000
mean,NaN,14861.175209,14861.175209,14856.369609,442.138013,7.491277e+04,NaN,18.041774,0.343849,0.754429,33.413089,70.704718,11.668204,0.222692,1.651187e+04,26.016001,0.015407,42.401605,1.0,NaN,NaN,NaN,0.005811,2.785309e+02,1.399818e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.068389e+04,NaN,NaN,NaN,4.402285,13434.192525,8474.474795,64.690304,0.010864,10.233105,128.504398,185.708692,13.088507,8.007239,1.853980,24.468987,39.644077,6.922530,35.498095,0.502935,3.689539,5.806444,4.651396,8.553835,8.566567,8.277823,15.307341,5.771242,11.618970,0.000972,0.003757,0.094148,2.006114,94.242178,50.813942,0.134636,0.056118,1.704131e+05,4.837393e+04,2.005252e+04,3.985870e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,117.013692,3.0,14.720930,346.626316,8074.341483,189.776919
std,NaN,8430.057833,8430.057833,8427.249607,244.835420,5.732357e+04,NaN,8.018367,0.895951,1.031650,21.799673,28.471355,5.269002,0.613581,2.097109e+04,11.898213,0.144975,20.861594,0.0,NaN,NaN,NaN,0.082312,2.117204e+04,1.531619e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.074759e+04,NaN,NaN,NaN,2.866401,16083.804107,13409.356259,26.411095,0.118429,588.295326,51.261958,93.019597,16.165945,8.773518,2.165458,30.335516,22.566049,5.927801,22.326416,1.263599,2.155459,3.140503,2.726511,4.826761,7.294576,4.319682,8.059532,3.122710,5.266977,0.032199,0.066103,0.487769,1.606979,8.466551,34.894723,0.375284,0.422999,1.712743e+05,4.580645e+04,2.029315e

In [13]:
# Let h2o load

train = h2o.load_dataset('C:\\Users\justi\OneDrive\桌面\Lending club\\train_lc')
test = h2o.load_dataset('C:\\Users\justi\OneDrive\桌面\Lending club\\test_lc')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
# Identify predictors and response
x = train.columns
y = "loan_status"
x.remove(y)


# For binary classification, response should be a factor
# make 2nd column a factor
train[y] = train[y].asfactor() 
test[y] = test[y].asfactor()

# Run AutoML 
aml = H2OAutoML(project_name='LC', 
                max_models=50,         # 50 base models
                balance_classes=True,  # Doing smart Class imbalance sampling
                max_runtime_secs=7200, # 2 hours
                seed=1234)             # Set a seed for reproducability
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
23:59:45.103: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

### What's the composition of the StackedEnsemble Leader model ?

In [ ]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
metalearner = h2o.get_model(aml.leader.metalearner()['name'])
metalearner.std_coef_plot()

In [ ]:
# You can see above H2O ML ran XGBoost, GLM, XRT, GBM, Deep Learning, DRF etc., did hyperparameter search and then
# combined the top N best algorithms to create a final stack ensemble

### Important Features and Attributes of the top GLM Model on the AutoML leaderboard
#### What's different about a  Loan getting "Fully Paid" or "Charged Off" ?

In [ ]:
# Get the GLM leader from above for looking at feature importance
df = lb.as_data_frame()
for m in df['model_id'].tolist():
    if (m.find("GLM") != -1):
        glm_leader_model = m
        break
        
h2o.get_model(glm_leader_model).std_coef_plot(num_of_features=20)

### Insights ...
The standardized Coef. Magnitudes of the GLM model in the leaderboard gives us a sense of what's different about a Loan Getting Paid in Full vs Loan getting Charged Off/Defaulted. The features/attributes in blue are the positive reasons (Length of the bar is the order of importance) why the Loan is getting Paid in Full vs the one in the Orange which can be attributed to Loan defaulting. In summary:

##### 7 Factors why Loan might getting Fully Paid - in the order of importance (Looking only at the Blue bars):

1. **term.36_months** - If the Loan term is shorter, like 3 years
2. **grade.A** - If the Loan Grade is "A"
3. **total_bc_limit** - If the total bank card credit Limit is high
4. **mo_sub_old_rev_tl_op** - If a lot of months since most recent revolving account opened
5. **home_ownership.MORTGAGE** - Whether a Home Mortgage Account was opened earlier by customer
6. **total_il_high_credit_limit** - Total installment high credit/credit limit (Kind of %payments to total credit limit)
7. **earliest_cr_line** - When the first credit line was opened


##### 7 Factors why Loan might be Charged Off - in the order of importance (Looking only at the Orange bars):
1. **int_rate** - If Interest Rate is high
2. **term.60_months** - If the Loan term is longer, like 5 years
3. **emp_title.<EMP_TITLE>** - 
4. **acc_open_past_24_mnts** - Correlated to the # of accounts opened in past 24 months
5. **dti** - Debt to Income ratio is high
6. **issue_d** - month/year which a loan was issued
7. **home_ownership.RENT** - Whether a customer was renting an apartment.

### Variable Importance Insights from each model in the AutoML Leaderboard

Note that each algorithm in the leaderboard is somewhat showing the same variables as the Top N important ones. That's telling us each algorithm generalizes to the same factors more or less !

In [ ]:
# Let's do variable importance for each one of the models in the LeaderBoard
# We can't do this for a StackedEnsemble, so will skip it

df = lb.as_data_frame()
for m in df['model_id'].tolist():
    if (m.find("StackedEnsemble") == -1):
        h2o.get_model(m).varimp_plot()

### Lets predict on the Test Data Set - using the AutoML leader 

In [ ]:
test_pc = aml.predict(test)

In [ ]:
# These scores were predicted by the stacked ensemble leader ...
test_pc

### AutoML Leader - Model Performance on the Test Data Set

In [ ]:
aml.leader.model_performance(test_data=test,xval=False)

### Summary of Results
The AUC on the test set was ~ 0.73, which is slightly better than the train AUC. 

The data was a snapshot on time where loans where running (some early stage and some late) and not necessarily "cohorts". In the data preparation phase, we also dropped lot of columns that was giving away the outcome. The models built are still very useful to understand the drivers behind the outcome. How to improve the accuracy ? See Next Steps.

### Next Steps
H2O3 AutoML can help you build models really quickly and understand the variable importances with very little effort. Recall, we didn't do any feature engineering (like one-hot-encoding etc.,) to the input data! In the next blog posts, we will explore how to do the following, in addition to Automatic Machine Learning:
- Automatic Feature Engineering
- Machine Learning Interpretability etc.,

with H2O's commercial product Driverless AI ...
